# BME-230B Spring 2019 HW 4 Question 
James Casaletto, Andrew Davidson, Yuanqing Xue, Jim Zheng

## <span style="color:red">TODO</span>
* 4.a does not ask for graph. its useful for debugging
* 4.b
* remove louvain debug cell
* add ref: URLS
* is this correct? did not fix the warning/errors from sc.tl.louvain and sc.tl.umap
    - see  euclid_bbknn.py _update_adata(self, l=None):
    ```
    n = self._neighbors_within_batch
    if l:
        n = self._numBatches * l

    self._adata.uns['neighbors']['params']['n_neighbors']=n
    ```

## 4.a. [10 pts] 
Turn in code that can compute a bb-l-k-NNG from a given bb-k-NNG. In addition to the input bb-k-NNG, the method should take in a parameter specifying the number of subsampled graphs that should be returned. Each of the returned subsampled graphs should represent a distinct sub-sampled bb-l-k-NNG from the given input graph. Fill in the l_k_bbknn method in euclid_bknn.py. You can set the l parameter accordingly. NOTE: in order to run this function successfully you will have to run the bbknn() method first, and then run the l_k_bbknn method. Consult example usage within euclid_bbknn.py.


In [ ]:
from euclid_bbknn import bbknn_graph
# import gseapy as gp
import matplotlib.pyplot as plt
# import numpy as np
# import os


#import pandas as pd
#import scanpy.api as sc
import scanpy as sc
print("scanpy.__version__:{}".format(sc.__version__))

In [ ]:
%%time
anndata = sc.read("PBMC.merged.h5ad")

In [ ]:
%%time
# run our implementation of nearest neighboors and update anndata
bb6nn = bbknn_graph(anndata, neighbors_within_batch=6, runPCA=True, pcs=50)

In [ ]:
%%time
sc.tl.louvain(anndata,
                  flavor='igraph', 
                  directed=False, 
                  use_weights=True)

In [ ]:
# save the bb6nn results. we will need them for 4.b
bb6nnConnectivities = anndata.uns['neighbors']['connectivities']
bb6nnDistances      = anndata.uns['neighbors']['distances']
bb6nnLouvain        = anndata.obs['louvain']

In [ ]:
# debug
bb6nnLouvain.head()

In [ ]:
# how to you print csr_matrix ?
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
print(type(bb6nnConnectivities))
print(bb6nnConnectivities.shape)
print(type(bb6nnConnectivities[10:15, 0:5]))
print(bb6nnConnectivities[0:5, 0:5])

In [ ]:
## <span style="color:red">AEDWIP</span>

In [ ]:
%%time
bb6nn.l_k_bbknn(l=3)

In [ ]:
%%time
sc.tl.louvain(anndata,
                  flavor='igraph', 
                  directed=False, 
                  use_weights=True)

In [ ]:
# save the bb3-6nn results. we will need them for 4.b
bb3_6nnConnectivities = anndata.uns['neighbors']['connectivities']
bb3_6nnDistances      = anndata.uns['neighbors']['distances']
bb3_6nnLouvain        = anndata.obs['louvain']

In [ ]:
%%time
sc.tl.umap(anndata)

In [ ]:
plt.figure(figsize=(10,10))
sc.pl.umap(anndata, color=["louvain"])

## 4.b. [10 pts] 
Turn in a bar plot of the Adjusted Rand Index (ARI) for Louvain clusters obtained from 10 independently subsampled bb-3-6-NNGs compared to the Louvain clusters obtained on the original bb-6-NNGs. Also report the average and standard deviations of the ARI. Based on these results, would you conclude these clusters are robust? Justify your answer. Hint: check if your ARI is significantly better than chance.


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import matplotlib.pyplot as plt

# number of subsamples to run
subsamples = 10

# list of ARIs obtained from each subsample
results = []

for sample in range(subsamples):
    bb3_6nn = bb6nn.l_k_bbknn(l=3)
    sc.tl.louvain(anndata, flavor='igraph', directed=False, use_weights=True)
    bb3_6nnConnectivities = anndata.uns['neighbors']['connectivities']
    bb3_6nnDistances      = anndata.uns['neighbors']['distances']
    bb3_6nnLouvain        = anndata.obs['louvain']
    ars = adjusted_rand_score(bb3_6nnLouvain, bb6nnLouvain)
    print('sample', sample, 'finished')
    results.append(ars)

# plot ARIs and print statistics
plt.bar(range(len(results)), results, color='black')
print('Average:', np.mean(results), 'SD:', np.std(results))